<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/Benchmarking_XGBoost_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Link to supporting video for below note book walkhthrough** - [https://www.youtube.com/watch?v=X7cKC6GgyxY](https://www.youtube.com/watch?v=X7cKC6GgyxY)

In [0]:
import xgboost as xgb
xgb.__version__

'0.90'

In [0]:
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)

In [0]:
covtyp.data.shape

(581012, 54)

In [0]:
np.unique(covtyp.target)

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [0]:
!nvidia-smi

Tue Jan  7 23:08:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

In [0]:
cov_df.memory_usage(index=True).sum()

255645408

In [0]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [0]:
print ("Rows     : " ,cov_df.shape[0])
print ("Columns  : " ,cov_df.shape[1])

Rows     :  581012
Columns  :  55


In [0]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [0]:
cov_df.dtypes

Elevation                             object
Aspect                                object
Slope                                 object
Horizontal_Distance_To_Hydrology      object
Vertical_Distance_To_Hydrology        object
Horizontal_Distance_To_Roadways       object
Hillshade_9am                         object
Hillshade_Noon                        object
Hillshade_3pm                         object
Horizontal_Distance_To_Fire_Points    object
Wilderness_Area1                      object
Wilderness_Area2                      object
Wilderness_Area3                      object
Wilderness_Area4                      object
Soil_Type1                            object
Soil_Type2                            object
Soil_Type3                            object
Soil_Type4                            object
Soil_Type5                            object
Soil_Type6                            object
Soil_Type7                            object
Soil_Type8                            object
Soil_Type9

In [0]:
for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols])

In [0]:
cov_df['target'] = cov_df['target']-1

In [0]:
cov_df_X = cov_df.copy()
cov_df_y =  cov_df_X.pop('target')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(cov_df_X, cov_df_y, train_size=0.85, test_size=0.15)

In [0]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [0]:
!nvidia-smi

Tue Jan  7 23:08:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import time
#setting tree and tree depth
num_round = 50
maxdepth = 8
param = {
  'colsample_bylevel': 1,
  'colsample_bytree': 1,
  'gamma': 0,
  'learning_rate': 0.1, 
  'random_state': 1010,
  'objective': 'multi:softmax', 
  'num_class': 7, 
}

In [0]:
param['tree_method'] = 'hist'
param['grow_policy'] = 'depthwise'
param['max_depth'] = maxdepth
param['max_leaves'] = 0
param['verbosity'] = 0
cpu_result = {} 
start_time = time.time()
# Training with the above parameters
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=cpu_result, verbose_eval=20)

print("CPU Training Time: %s seconds" % (str(time.time() - start_time)))

[0]	test-merror:0.220638
[20]	test-merror:0.183163
[40]	test-merror:0.168591
[49]	test-merror:0.163003
CPU Training Time: 80.21812081336975 seconds


In [0]:
param['tree_method'] = 'gpu_hist'
param['grow_policy'] = 'depthwise'
param['max_depth'] = maxdepth
param['max_leaves'] = 0
param['verbosity'] = 0
param['gpu_id'] = 0
param['updater'] = 'grow_gpu_hist'
param['predictor'] = 'gpu_predictor'

gpu_result = {} 
start_time = time.time()
# Training with the above parameters
xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')], evals_result=gpu_result, verbose_eval=20)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

[0]	test-merror:0.220683
[20]	test-merror:0.183048
[40]	test-merror:0.169715
[49]	test-merror:0.162934
GPU Training Time: 8.568038940429688 seconds


In [0]:
!nvidia-smi

Tue Jan  7 23:09:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    38W / 250W |    543MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  